In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Import Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 데이터셋 합치기

In [ ]:
# Read Data
cj = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/CJ/CJ.xlsx', error_bad_lines=False)
gs = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/GS/GS.csv',  encoding='cp949')
scr =  pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SC/CARD_SPENDING_RESIDENT.txt', sep='\t')
skt =  pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SK/skt.csv', index_col='Unnamed: 0')
ska = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SK/ska.csv', index_col='Unnamed: 0')

In [ ]:
# scr
scr.rename(columns = {'STD_DD' : 'STD_YMD'}, inplace = True)
scr.rename(columns = {'DONG_CD' : 'HDONG_CD'}, inplace = True)

scr['STD_YMD'] = pd.to_datetime(scr['STD_YMD'].astype(str), format='%Y%m%d')
scr = scr.drop(['GU_CD', 'SEX_CD', 'AGE_CD'], axis=1) 

def name_a_to_b(data,col,a,b):
  data = data.replace({col:a}, {col:b}, inplace=True)
  return(data)

cat_code = [10,20,21,22,30,31,32,33,34,35,40,42,43,44,50,52,60,62,70,71,80,81,92]
cat_name = ['숙박','레저용품','레저업소','문화취미','가구','전기','주방용구',
            '연료판매','광학제품','가전','유통업','의복','직물','신변잡화','서적문구','사무통신',
            '자동차판매','자동차정비','의료기관','보건위생','요식업소','음료식품','수리서비스']
            
for i in range(len(cat_code)):
  name_a_to_b(scr,'MCT_CAT_CD',cat_code[i],cat_name[i])

scr = scr.groupby(['STD_YMD', 'HDONG_CD', 'MCT_CAT_CD']).sum().unstack(level=2).fillna(0)

In [ ]:
# gs
gs.rename(columns = {'OPER_DT' : 'STD_YMD'}, inplace = True)
gs.rename(columns = {'ADMD' : 'HDONG_NM'}, inplace = True)

gs['STD_YMD'] = pd.to_datetime(gs['STD_YMD'].astype(str), format='%Y%m%d')

gs = gs.drop(['PVN', 'BOR'], axis=1)

In [ ]:
cj.rename(columns = {'DL_YMD' : 'STD_YMD'}, inplace = True)
cj.rename(columns = {'DL_GD_LCLS_NM' : 'CJ_CATG'}, inplace = True)
cj.rename(columns = {'HDNG_NM' : 'HDONG_NM'}, inplace = True)
cj.rename(columns = {'HDNG_CD' : 'HDONG_CD'}, inplace = True)

cj['STD_YMD'] = pd.to_datetime(cj['STD_YMD'].astype(str), format='%y%m%d')

cj['HDONG_NM'] = cj['HDONG_NM'].apply(lambda x: '신당5동' if x == '신당제5동' else x)

cj = cj.drop(['DL_GD_LCLS_CD', 'CTPV_NM', 'CTPV_CD', 'CTGG_NM', 'CTGG_CD'], axis=1)
cj = cj.set_index(['STD_YMD', 'HDONG_CD', 'HDONG_NM', 'CJ_CATG']).unstack(level=3).fillna(0)
cj = cj.reset_index(drop=False).set_index(['STD_YMD', 'HDONG_CD'])
cj.head()

HDONG_NM INVC_CONT               ...                         
CJ_CATG                        가구/인테리어 도서/음반 디지털/가전  ... 출산/육아  패션의류  패션잡화 화장품/미용
STD_YMD    HDONG_CD                                  ...                         
2019-02-01 510          범어1동       6.0  64.0   15.0  ...   0.0  29.0  12.0   19.0
           517           동인동       6.0  41.0    6.0  ...   1.0  21.0   9.0   21.0
           520          범어2동      12.0  53.0   14.0  ...   0.0  36.0  19.0   28.0
           520           소공동       3.0  48.0   35.0  ...   0.0  25.0  10.0   26.0
           530          범어3동       4.0  46.0   10.0  ...   1.0  39.0  11.0   22.0

[5 rows x 11 columns]

In [ ]:
skt['STD_YMD'] = pd.to_datetime(skt['STD_YMD'].astype(str), format='%Y%m%d') # dtype datetime으로 변경

skt['TMST_MORNING'] = np.mean(skt.iloc[:,15:19], axis=1)
skt['TMST_NOON'] = np.mean(skt.iloc[:,15:18], axis=1)
skt['TMST_AFTNOON'] = np.mean(skt.iloc[:,18:22], axis=1)
skt['TMST_NIGHT'] = np.mean(skt.iloc[:,22:26], axis=1)
skt['TMST_DAWN'] = np.mean(pd.concat([skt.iloc[:, 26:28], skt.iloc[:, 5:11]], axis=1), axis=1)

skt = skt[['STD_YMD', 'HDONG_NM', 'TMST_MORNING', 'TMST_NOON', 'TMST_AFTNOON', 'TMST_NIGHT', 'TMST_DAWN']]


# skt = skt.set_index(['HDONG_NM', 'STD_YMD']) # index로 설정

# ska_backup = ska
# ska['STD_YMD'] = pd.to_datetime(ska['STD_YMD'].astype(str), format='%Y%m%d') # dtype datetime으로 변경
# ska = ska.set_index(['HDONG_NM', 'STD_YMD']) # index로 설정

# sk = pd.concat([skt, ska.iloc[:,3:]], axis=1)

In [ ]:
# cj랑 sc 코드로 합치고, 코드 지우기
all = pd.merge(cj, scr, left_on=['STD_YMD', 'HDONG_CD'], right_on=['STD_YMD', 'HDONG_CD'], how='left')
all = all.reset_index(drop=False).set_index(['STD_YMD', 'HDONG_NM']).drop('HDONG_CD', axis=1)
all = pd.merge(all, gs, left_on=['STD_YMD', 'HDONG_NM'], right_on=['STD_YMD', 'HDONG_NM'], how='left')
all = pd.merge(all, skt, left_on=['STD_YMD', 'HDONG_NM'], right_on=['STD_YMD', 'HDONG_NM'], how='left')

In [ ]:
#all.to_csv('./gdrive/My Drive/빅콘 대상팀/data/all.csv', sep=',',index=False)

In [ ]:
all = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/all.csv')

In [ ]:
gu = pd.read_excel('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/구_동.xlsx')

In [ ]:
all.columns =[ 'STD_YMD', 'HDONG_NM', 'CJ_가구/인테리어',
       'CJ_도서/음반', 'CJ_디지털/가전',
       'CJ_생활건강', 'CJ_스포츠/레저',
       'CJ_식품', 'CJ_출산/육아',
       'CJ_패션의류', 'CJ_패션잡화',
       'CJ_화장품/미용', 'SC_CNT_가구', 'SC_CNT_가전',
       'SC_CNT_광학제품', 'SC_CNT_레저업소', 'SC_CNT_레저용품',
       'SC_CNT_문화취미', 'SC_CNT_보건위생', 'SC_CNT_사무통신',
       'SC_CNT_서적문구', 'SC_CNT_수리서비스', 'SC_CNT_숙박',
       'SC_CNT_신변잡화', 'SC_CNT_연료판매', 'SC_CNT_요식업소',
       'SC_CNT_유통업', 'SC_CNT_음료식품', 'SC_CNT_의료기관',
       'SC_CNT_의복', 'SC_CNT_자동차정비', 'SC_CNT_자동차판매',
       'SC_CNT_전기', 'SC_CNT_주방용구', 'SC_CNT_직물',
       'SC_AMT_가구', 'SC_AMT_가전', 'SC_AMT_광학제품',
       'SC_AMT_레저업소', 'SC_AMT_레저용품', 'SC_AMT_문화취미',
       'SC_AMT_보건위생', 'SC_AMT_사무통신', 'SC_AMT_서적문구',
       'SC_AMT_수리서비스', 'SC_AMT_숙박', 'SC_AMT_신변잡화',
       'SC_AMT_연료판매', 'SC_AMT_요식업소', 'SC_AMT_유통업',
       'SC_AMT_음료식품', 'SC_AMT_의료기관', 'SC_AMT_의복',
       'SC_AMT_자동차정비', 'SC_AMT_자동차판매', 'SC_AMT_전기',
       'SC_AMT_주방용구', 'SC_AMT_직물', 'GS_매출지수', 'GS_식사',
       'GS_간식', 'GS_마실거리', 'GS_홈/리빙', 'GS_헬스/뷰티', 'GS_취미/여가활동',
       'GS_사회활동', 'GS_임신육아', 'TMST_MORNING', 'TMST_NOON', 'TMST_AFTNOON',
       'TMST_NIGHT', 'TMST_DAWN']

In [ ]:
data_final = pd.merge(all,gu,how='left',on = 'HDONG_NM')

In [ ]:
#data_final.to_csv('./gdrive/My Drive/빅콘 대상팀/data/all_clean.csv', sep=',',index=False)

In [ ]:
all = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/all_clean.csv' ,parse_dates=['STD_YMD'])

In [ ]:
for i in np.arange(-14,-6,1):
  all.iloc[:,i] = all.iloc[:,i].str.replace('%','').astype(float)

In [ ]:
all[all.filter(like='GS').columns[1:]] = all[all.filter(like='GS').columns[1:]].multiply(all["GS_매출지수"], axis="index")

In [ ]:
#all.to_csv('./gdrive/My Drive/빅콘 대상팀/data/all_amt.csv', sep=',',index=False)